In [3]:
import requests
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from pathlib import Path
import re
import csv
from bs4 import BeautifulSoup
import pandas as pd
import os
import requests
import pandas as pd
import time
from pathlib import Path
import os
import asyncio
from pathlib import Path
import pandas as pd
import aiohttp
import nest_asyncio

In [ ]:
!rm -rf data/

In [ ]:
# Download Players Name/IDs from https://www.mlb.com/players

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=options)

url = "https://www.mlb.com/players"
driver.get(url)
time.sleep(10)
html = driver.page_source
output_path = "data/mlb_players_raw.html"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with open(output_path, "w", encoding="utf-8") as f:
    f.write(html)
driver.quit()
print("Downloaded full HTML with JavaScript saved to", output_path)

# Parse All Player ID's
html_path = "data/mlb_players_raw.html"
output_csv = "data/mlb_players.csv"
with open(html_path, "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")
pattern = re.compile(r"^/player/([a-z0-9\-]+-\d+)$", re.IGNORECASE)
anchors = soup.find_all("a", class_="p-related-links__link")
player_entries = []
for a in anchors:
    href = a.get("href", "")
    text = a.get_text(strip=True)
    match = pattern.match(href)
    if match:
        raw_slug = match.group(1)
        player_entries.append([raw_slug, text])
unique = {slug: name for slug, name in player_entries}
unique_entries = [[slug, name] for slug, name in unique.items()]
with open(output_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Player ID", "Player Name"])
    writer.writerows(unique_entries)
df = pd.read_csv(output_csv)
print("Sample of extracted player data:")
print(df.head(20))
print(f"Total unique players extracted: {df.shape[0]}")

In [ ]:
# # Web Version Directly ^

# url = "https://www.mlb.com/players"
# output_csv = "data/mlb_players.csv"

# chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--disable-gpu")
# # chrome_options.add_argument("--window-size=1920,1080")
# driver = webdriver.Chrome(options=chrome_options)
# print("Loading the page...")
# driver.get(url)
# time.sleep(10)
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# time.sleep(5)
# print("Extracting player link elements...")

# player_elements = driver.find_elements(By.CSS_SELECTOR, "a.p-related-links__link")
# pattern = re.compile(r"/player/([a-z0-9\-]+-\d+)", re.IGNORECASE)
# player_data = []
# for elem in player_elements:
#     href = elem.get_attribute("href")
#     text = elem.text.strip()
#     match = pattern.search(href)
#     if match:
#         raw_slug = match.group(1)
#         player_data.append([raw_slug, text])
# print(f"Found {len(player_data)} player entries.")
# driver.quit()  # Close the browser

# # --- Write the extracted data to CSV ---
# print(f"Writing data to {output_csv}...")
# with open(output_csv, "w", newline="", encoding="utf-8") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Player ID", "Player Name"])
#     writer.writerows(player_data)

# # --- Preview the results using pandas ---
# df = pd.read_csv(output_csv)
# print("Sample of extracted player data:")
# print(df.head(20))
# print(f"Total unique players extracted: {df.shape[0]}")


In [ ]:
# Download Player Batting Pages
# From baseballsavant.mlb.com/savant-player<player-id>?stats=statcast-r-hitting-mlb

# Download player batting pages
nest_asyncio.apply()
csv_path = "data/mlb_players.csv"
output_folder = Path("data/raw/players-batting")
output_folder.mkdir(parents=True, exist_ok=True)
df = pd.read_csv(csv_path)
async def fetch(session, url, raw_slug, player_name, max_retries=3):
    for attempt in range(1, max_retries + 1):
        try:
            print(f"Fetching: {player_name} -> {url} (Attempt {attempt})")
            async with session.get(url, timeout=15) as response:
                if response.status == 200:
                    text = await response.text()
                    output_file = output_folder / f"{raw_slug}.html"
                    with open(output_file, "w", encoding="utf-8") as f:
                        f.write(text)
                    print(f"Successfully fetched {player_name}")
                    break
                else:
                    print(f"⚠️ Failed: {player_name} (HTTP {response.status}).")
        except Exception as e:
            print(f"❌ Error fetching {url} on attempt {attempt}: {e}")
        if attempt < max_retries:
            await asyncio.sleep(1)  # Wait a bit before retrying
        else:
            print(f"❌ Giving up on {player_name} after {max_retries} attempts.")

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for _, row in df.iterrows():
            raw_slug = row["Player ID"]     # e.g., "mike-yastrzemski-573262"
            player_name = row["Player Name"]
            url = f"https://baseballsavant.mlb.com/savant-player/{raw_slug}?stats=statcast-r-hitting-mlb"
            tasks.append(fetch(session, url, raw_slug, player_name))
        await asyncio.gather(*tasks)
if __name__ == "__main__":
    asyncio.run(main())
    
# v2 #
# csv_path = "data/mlb_players.csv"
# output_folder = Path("data/raw/players-batting")
# output_folder.mkdir(parents=True, exist_ok=True)
# df = pd.read_csv(csv_path)
# options = Options()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
# driver = webdriver.Chrome(options=options)
# skipped = 0
# downloaded = 0
# for _, row in df.iterrows():
#     raw_slug = row["Player ID"]
#     player_name = row["Player Name"]
#     output_file = output_folder / f"{raw_slug}.html"
#     if output_file.exists():
#         print(f"Skipping: {player_name} (already downloaded)")
#         skipped += 1
#         continue
        
#     url = f"https://baseballsavant.mlb.com/savant-player/{raw_slug}?stats=statcast-r-hitting-mlb"
#     try:
#         print(f"Fetching: {player_name} -> {url}")
#         driver.get(url)
#         html = driver.page_source
#         with open(output_file, "w", encoding="utf-8") as f:
#             f.write(html)
#         downloaded += 1
#     except Exception as e:
#         print(f"❌ Error fetching {url}: {e}")
# driver.quit()
# print(f"Completed downloading {len(df)} player pages to {output_folder}")

# v0 #
# csv_path = "data/mlb_players.csv"
# output_folder = Path("data/raw/players-batting")
# output_folder.mkdir(parents=True, exist_ok=True)
# df = pd.read_csv(csv_path)
# for idx, row in df.iterrows():
#     raw_slug = row["Player ID"]  # e.g., "mike-yastrzemski-573262"
#     player_name = row["Player Name"]

#     url = f"https://baseballsavant.mlb.com/savant-player/{raw_slug}?stats=statcast-r-hitting-mlb"
#     print(f"[{idx+1}/{len(df)}] Fetching: {player_name} -> {url}")

#     try:
#         response = requests.get(url, timeout=15)
#         if response.status_code == 200:
#             output_file = output_folder / f"{raw_slug}.html"
#             with open(output_file, "w", encoding="utf-8") as f:
#                 f.write(response.text)
#         else:
#             print(f"⚠️ Failed: {player_name} (HTTP {response.status_code})")
#     except requests.RequestException as e:
#         print(f"❌ Error fetching {url}: {e}")
#     time.sleep(0.2)

In [4]:
### Parse For General Player Info ###

input_folder = Path("data/raw/players-batting")
output_csv = "data/player_general_info.csv"
rows = []
all_files = list(input_folder.glob("*.html"))
total_files = len(all_files)
processed = 0
print(f"Processing {total_files} player files...")

# Change the loop to use all_files
# for file in input_folder.glob("*.html"):
for file in all_files:
    processed += 1
    if processed % 20 == 0 or processed == total_files:
        print(f"Progress: {processed}/{total_files} files ({processed/total_files*100:.1f}%)")
    with open(file, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
    bio_section = soup.find("div", class_="bio-player-name")
    if not bio_section:
        continue
    player_name = bio_section.find("div").get_text(strip=True)
    subsections = bio_section.find_all("div", style=lambda v: v and "font-size: .8rem" in v)
    if len(subsections) < 1:
        continue
    line1 = subsections[0].get_text(" ", strip=True)
    parts = [p.strip() for p in line1.split("|")]
    if len(parts) < 4:
        continue
    position = parts[0]
    bats_throws = parts[1].replace("Bats/Throws:", "").strip()
    height_weight = parts[2].strip()
    age = parts[3].replace("Age:", "").strip()
    hw_parts = height_weight.split()
    height = hw_parts[0] + " " + hw_parts[1]
    weight = hw_parts[2] if len(hw_parts) > 2 else ""
    player_id = file.stem
    rows.append({
        "Player ID": player_id,
        "Position": position,
        "Bats/Throws": bats_throws,
        "Height": height,
        "Weight": weight,
        "Age": age
    })
df = pd.DataFrame(rows)
df.to_csv(output_csv, index=False)
print(f"✅ Saved {len(df)} players to {output_csv}")

!open data/player_general_info.csv

Processing 1454 player files...
Progress: 20/1454 files (1.4%)
Progress: 40/1454 files (2.8%)
Progress: 60/1454 files (4.1%)
Progress: 80/1454 files (5.5%)
Progress: 100/1454 files (6.9%)
Progress: 120/1454 files (8.3%)
Progress: 140/1454 files (9.6%)
Progress: 160/1454 files (11.0%)
Progress: 180/1454 files (12.4%)
Progress: 200/1454 files (13.8%)
Progress: 220/1454 files (15.1%)
Progress: 240/1454 files (16.5%)
Progress: 260/1454 files (17.9%)
Progress: 280/1454 files (19.3%)
Progress: 300/1454 files (20.6%)
Progress: 320/1454 files (22.0%)
Progress: 340/1454 files (23.4%)
Progress: 360/1454 files (24.8%)
Progress: 380/1454 files (26.1%)
Progress: 400/1454 files (27.5%)
Progress: 420/1454 files (28.9%)
Progress: 440/1454 files (30.3%)
Progress: 460/1454 files (31.6%)
Progress: 480/1454 files (33.0%)
Progress: 500/1454 files (34.4%)
Progress: 520/1454 files (35.8%)
Progress: 540/1454 files (37.1%)
Progress: 560/1454 files (38.5%)
Progress: 580/1454 files (39.9%)
Progress: 600/1454 fil

In [6]:
### Parse for Batting Statistics Table ###

input_folder = Path("data/raw/players-batting")
output_csv = "data/player_batting_stats.csv"
all_rows = []

all_files = list(input_folder.glob("*.html"))
total_files = len(all_files)
processed = 0
success = 0
print(f"Processing {total_files} player files for batting stats...")
for file_path in all_files:
    processed += 1
    if processed % 20 == 0 or processed == total_files:
        print(f"Progress: {processed}/{total_files} files ({processed/total_files*100:.1f}%)")
        
    with open(file_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
    slug = file_path.stem
    table_div = soup.find("div", id="statcast_glance_batter")
    if not table_div:
        continue
    table = table_div.find("table")
    if not table:
        continue
    headers = [th.get_text(strip=True).replace("\n", " ") for th in table.find("thead").find_all("th")]
    for tr in table.find("tbody").find_all("tr"):
        cells = tr.find_all("td")
        row_data = [td.get_text(strip=True) for td in cells]
        if len(row_data) == len(headers):
            row_dict = dict(zip(headers, row_data))
            row_dict["Player ID"] = slug
            all_rows.append(row_dict)
            success += 1

df = pd.DataFrame(all_rows)
df = df[~df['Season'].isin(['Player', 'MLB'])]
df.to_csv(output_csv, index=False)
print(f"✅ Saved {len(df)} rows to {output_csv}")

!open data/player_batting_stats.csv

# for file_path in input_folder.glob("*.html"):
#     with open(file_path, "r", encoding="utf-8") as f:
#         soup = BeautifulSoup(f, "html.parser")
#     slug = file_path.stem
#     table_div = soup.find("div", id="statcast_glance_batter")
#     if not table_div:
#         continue
#     table = table_div.find("table")
#     if not table:
#         continue
#     headers = [th.get_text(strip=True).replace("\n", " ") for th in table.find("thead").find_all("th")]
#     for tr in table.find("tbody").find_all("tr"):
#         cells = tr.find_all("td")
#         row_data = [td.get_text(strip=True) for td in cells]
#         if len(row_data) == len(headers):
#             row_dict = dict(zip(headers, row_data))
#             row_dict["Player Slug"] = slug
#             all_rows.append(row_dict)
# df = pd.DataFrame(all_rows)
# df.to_csv(output_csv, index=False)
# print(f"✅ Saved {len(df)} rows to {output_csv}")


Processing 1454 player files for batting stats...
Progress: 20/1454 files (1.4%)
Progress: 40/1454 files (2.8%)
Progress: 60/1454 files (4.1%)
Progress: 80/1454 files (5.5%)
Progress: 100/1454 files (6.9%)
Progress: 120/1454 files (8.3%)
Progress: 140/1454 files (9.6%)
Progress: 160/1454 files (11.0%)
Progress: 180/1454 files (12.4%)
Progress: 200/1454 files (13.8%)
Progress: 220/1454 files (15.1%)
Progress: 240/1454 files (16.5%)
Progress: 260/1454 files (17.9%)
Progress: 280/1454 files (19.3%)
Progress: 300/1454 files (20.6%)
Progress: 320/1454 files (22.0%)
Progress: 340/1454 files (23.4%)
Progress: 360/1454 files (24.8%)
Progress: 380/1454 files (26.1%)
Progress: 400/1454 files (27.5%)
Progress: 420/1454 files (28.9%)
Progress: 440/1454 files (30.3%)
Progress: 460/1454 files (31.6%)
Progress: 480/1454 files (33.0%)
Progress: 500/1454 files (34.4%)
Progress: 520/1454 files (35.8%)
Progress: 540/1454 files (37.1%)
Progress: 560/1454 files (38.5%)
Progress: 580/1454 files (39.9%)
Prog

In [2]:
### Parse for Pitch Tracking Table ###

# start:
# /html/body/div[2]/section/div/section/div[5]/section/div[1]/div[6]/div[2]/div/table/thead
# /html/body/div[2]/section/div/section/div[5]/section/div[1]/div[6]/div[2]/div/table/thead/tr

# column 1:
# /html/body/div[2]/section/div/section/div[5]/section/div[1]/div[6]/div[2]/div/table/thead/tr/th[1]

# columns 2:
# /html/body/div[2]/section/div/section/div[5]/section/div[1]/div[6]/div[2]/div/table/thead/tr/th[2]

# body:
# /html/body/div[2]/section/div/section/div[5]/section/div[1]/div[6]/div[2]/div/table/tbody

# row 1 values inside:
# /html/body/div[2]/section/div/section/div[5]/section/div[1]/div[6]/div[2]/div/table/tbody/tr[1]

# column 1 value
# /html/body/div[2]/section/div/section/div[5]/section/div[1]/div[6]/div[2]/div/table/thead/tr/th[1]
# /html/body/div[2]/section/div/section/div[5]/section/div[1]/div[6]/div[2]/div/table/thead/tr/th[1]/div

# last column value
# /html/body/div[2]/section/div/section/div[5]/section/div[1]/div[6]/div[2]/div/table/thead/tr/th[23]
# /html/body/div[2]/section/div/section/div[5]/section/div[1]/div[6]/div[2]/div/table/thead/tr/th[23]/div


# that is not the correct table for the last fucking time. This Pitch Tracking one is with these columns in the image ONLY 
# Year, Pitch Type, #, %, PA, AB, H, 1B, 2B, 3B, HR, SO, BBE, BA, XBA, SLG, XSLG, WOBA, XWOBA, EV, LA, Whiff%, PutAway%

# Pitch Tracking
# 2025	Fastball	144	49.0	39	36	11	8	0	0	3	5	31	.306	.344	.556	.752	.399	.477	95.9	12	15.9	17.9
# 2025	Breaking	104	35.4	25	21	5	3	1	1	0	9	12	.238	.290	.381	.449	.335	.378	97.3	10	39.2	22.0
# 2025	Offspeed	46	15.6	11	9	2	1	0	0	1	4	5	.222	.280	.556	.550	.399	.413	97.3	7	42.9	33.3
# 2024	Fastball	1,520	53.6	395	338	105	59	23	2	21	73	268	.311	.326	.577	.636	.414	.444	95.7	15	24.9	20.4
# 2024	Breaking	866	30.5	216	195	55	21	9	2	23	67	129	.282	.275	.703	.665	.435	.425	95.0	20	38.1	23.0
# 2024	Offspeed	452	15.9	110	103	37	18	6	3	10	22	82	.359	.348	.767	.731	.480	.468	96.8	14	29.5	17.6



# input_folder = Path("data/raw/players-batting")
# output_csv = "data/player_pitch_tracking_stats.csv"
# all_pitch_rows = []
# for file_path in input_folder.glob("*.html"):
#     with open(file_path, "r", encoding="utf-8") as f:
#         soup = BeautifulSoup(f, "html.parser")
#     slug = file_path.stem
#     table = soup.find("table", id="detailedPitches")
#     if not table:
#         continue
#     headers = [th.get_text(strip=True).replace("\n", " ") for th in table.find("thead").find_all("th")]
#     for tr in table.find("tbody").find_all("tr"):
#         tds = tr.find_all("td")
#         row = [td.get_text(strip=True) for td in tds]
#         if len(row) == len(headers):
#             row_dict = dict(zip(headers, row))
#             row_dict["Player Slug"] = slug
#             all_pitch_rows.append(row_dict)
# df_pitch = pd.DataFrame(all_pitch_rows)
# df_pitch.to_csv(output_csv, index=False)
# print(f"✅ Saved {len(df_pitch)} rows to {output_csv}")




# input_folder = Path("data/raw/players-batting")
# output_csv = "data/player_pitch_tracking_stats.csv"
# all_rows = []

# # Count total files first
# all_files = list(input_folder.glob("*.html"))
# total_files = len(all_files)
# processed = 0
# success = 0

# print(f"Processing {total_files} player files for pitch tracking stats...")

# for file_path in all_files:
#     processed += 1
#     if processed % 20 == 0 or processed == total_files:
#         print(f"Progress: {processed}/{total_files} files ({processed/total_files*100:.1f}%)")
    
#     with open(file_path, "r", encoding="utf-8") as f:
#         soup = BeautifulSoup(f, "html.parser")
    
#     slug = file_path.stem
#     table = soup.find("table", {"id": "detailedPitches"})
#     if not table:
#         continue

#     # Extract column headers
#     headers = [th.get_text(strip=True).replace("\n", " ") for th in table.find("thead").find_all("th")]

#     # Extract data rows
#     for tr in table.find("tbody").find_all("tr"):
#         cells = tr.find_all("td")
#         row_data = [td.get_text(strip=True).replace(",", "") for td in cells]
#         if len(row_data) == len(headers):
#             row_dict = dict(zip(headers, row_data))
#             row_dict["Player Slug"] = slug
#             all_rows.append(row_dict)
#             success += 1

# df = pd.DataFrame(all_rows)
# df.to_csv(output_csv, index=False)
# print(f"✅ Saved {len(df)} rows to {output_csv}")

### Parse for Pitch Tracking Table ###



# import os
# import pandas as pd
# from bs4 import BeautifulSoup
# from pathlib import Path
# from tqdm import tqdm

# input_folder = Path("data/raw/players-batting")
# output_csv = Path("data/player_pitch_tracking_stats.csv")
# all_rows = []

# all_files = list(input_folder.glob("*.html"))
# total_files = len(all_files)

# print(f"\n🔍 Scanning {input_folder} for player HTML files...\n")

# for idx, file_path in enumerate(tqdm(all_files, desc="Processing Players")):
#     slug = file_path.stem
#     with open(file_path, "r", encoding="utf-8") as f:
#         soup = BeautifulSoup(f.read(), "html.parser")

#     # Try to find the <h2> heading for Pitch Tracking
#     pitch_heading = soup.find("h2", string=lambda s: s and "Pitch Tracking" in s)
#     if not pitch_heading:
#         print(f"❌ {slug}: 'Pitch Tracking' section not found")
#         continue

#     # Look for the table after the heading
#     table = None
#     next_tag = pitch_heading.find_next_sibling()
#     while next_tag:
#         if next_tag.name == "div" and next_tag.find("table"):
#             table = next_tag.find("table")
#             break
#         next_tag = next_tag.find_next_sibling()

#     if not table:
#         print(f"❌ {slug}: Pitch Tracking table not found")
#         continue

#     # Extract headers
#     header_cells = table.find("thead").find_all("th")
#     headers = [cell.get_text(strip=True) for cell in header_cells]

#     # Extract data rows
#     for row in table.find("tbody").find_all("tr"):
#         cells = row.find_all("td")
#         if not cells:
#             continue
#         values = [td.get_text(strip=True) for td in cells]
#         if len(values) != len(headers):
#             continue
#         row_dict = dict(zip(headers, values))
#         row_dict["Player Slug"] = slug
#         all_rows.append(row_dict)

# # Save output to CSV
# df = pd.DataFrame(all_rows)
# df.to_csv(output_csv, index=False)
# print(f"\n✅ Done. Extracted {len(df)} total rows into {output_csv}")



# ASYNCIO VERSION # 


# import asyncio
# import aiohttp
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from bs4 import BeautifulSoup
# import pandas as pd
# from pathlib import Path
# from tqdm import tqdm
# import time
# import nest_asyncio
# from concurrent.futures import ThreadPoolExecutor
# from functools import partial

# # Apply nest_asyncio to allow async in Jupyter
# nest_asyncio.apply()

# # Setup paths and read player data
# df_players = pd.read_csv("data/mlb_players.csv")
# output_csv = "data/player_detailed_pitches.csv"
# all_rows = []

# # Setup Chrome options optimized for M1/M2/M4
# options = Options()
# options.add_argument("--headless=new")  # New headless mode
# options.add_argument("--disable-gpu")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")
# options.add_argument("--disable-features=TranslateUI")
# options.add_argument("--disable-extensions")
# options.add_argument("--disable-component-extensions-with-background-pages")
# options.add_argument("--disable-default-apps")
# options.add_argument("--enable-features=NetworkService,NetworkServiceInProcess")

# # Configure chunk size for parallel processing
# CHUNK_SIZE = 5  # Process 5 players simultaneously

# async def process_player(player_id, player_name, driver):
#     try:
#         url = f"https://baseballsavant.mlb.com/savant-player/{player_id}?stats=statcast-r-hitting-mlb"
        
#         driver.get(url)
#         await asyncio.sleep(1)  # Reduced wait time
        
#         soup = BeautifulSoup(driver.page_source, "html.parser")
        
#         # Find table by its specific ID
#         table = soup.find("table", id="detailedPitches")
#         if not table:
#             print(f"No pitch data found for {player_id}")
#             return []
            
#         # Extract headers
#         headers = []
#         for th in table.find("thead").find_all("th"):
#             header = th.get_text(strip=True)
#             header = header.replace("\n", " ").strip()
#             headers.append(header)
        
#         player_rows = []
#         # Process each row
#         for tr in table.find("tbody").find_all("tr"):
#             cells = tr.find_all("td")
#             if not cells:
#                 continue
                
#             row_data = {"Player ID": player_id}
            
#             for idx, cell in enumerate(cells):
#                 if idx < len(headers):
#                     value = cell.get_text(strip=True).replace(",", "")
#                     row_data[headers[idx]] = value
                    
#             if len(row_data) > 1:
#                 player_rows.append(row_data)
                
#         return player_rows
        
#     except Exception as e:
#         print(f"❌ Error processing {player_id}: {str(e)}")
#         return []

# async def process_chunk(chunk_df, chunk_idx):
#     driver = webdriver.Chrome(options=options)
#     chunk_rows = []
    
#     try:
#         for _, row in chunk_df.iterrows():
#             player_rows = await process_player(row["Player ID"], row["Player Name"], driver)
#             chunk_rows.extend(player_rows)
            
#         return chunk_rows
#     finally:
#         driver.quit()

# async def main():
#     chunks = [df_players[i:i + CHUNK_SIZE] for i in range(0, len(df_players), CHUNK_SIZE)]
#     print(f"\n🔍 Processing {len(df_players)} players in {len(chunks)} chunks...\n")
    
#     all_rows = []
    
#     for chunk_idx, chunk_df in enumerate(tqdm(chunks, desc="Processing Chunks")):
#         chunk_rows = await process_chunk(chunk_df, chunk_idx)
#         all_rows.extend(chunk_rows)
        
#         # Save progress after each chunk
#         if chunk_rows:
#             temp_df = pd.DataFrame(all_rows)
#             first_cols = ["Player ID", "Year", "Pitch Type"]
#             other_cols = [col for col in temp_df.columns if col not in first_cols]
#             temp_df = temp_df[first_cols + sorted(other_cols)]
#             temp_df.to_csv(output_csv, index=False)
#             print(f"\n💾 Progress saved: {len(all_rows)} total rows")
    
#     return all_rows

# if __name__ == "__main__":
#     try:
#         all_rows = asyncio.run(main())
        
#         if all_rows:
#             df = pd.DataFrame(all_rows)
#             first_cols = ["Player ID", "Year", "Pitch Type"]
#             other_cols = [col for col in df.columns if col not in first_cols]
#             df = df[first_cols + sorted(other_cols)]
#             df.to_csv(output_csv, index=False)
#             print(f"\n✅ Saved {len(df)} rows to {output_csv}")
#         else:
#             print("\n⚠️ No data was collected")
            
#     except KeyboardInterrupt:
#         print("\n\n⚠️ Process interrupted by user. Saving collected data...")
#         if all_rows:
#             df = pd.DataFrame(all_rows)
#             df.to_csv(output_csv, index=False)
#             print(f"Saved {len(df)} rows to {output_csv}")


import asyncio
import aiohttp
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import time
import nest_asyncio
from concurrent.futures import ThreadPoolExecutor
import json
from typing import List, Dict

# Apply nest_asyncio to allow async in Jupyter
nest_asyncio.apply()

# Setup paths and read player data
df_players = pd.read_csv("data/mlb_players.csv")
output_csv = "data/player_detailed_pitches.csv"
all_rows = []
players_processed = 0

# Define the exact columns we want in order
COLUMNS = [
    "Player ID", "Year", "Pitch Type", "#", "%", "PA", "AB", "H", "1B", "2B", 
    "3B", "HR", "SO", "BBE", "BA", "XBA", "SLG", "XSLG", "WOBA", "XWOBA", 
    "EV", "LA", "Whiff%", "PutAway%"
]

# Setup Chrome options optimized for M1/M2/M4
options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-features=TranslateUI")
options.add_argument("--disable-extensions")
options.add_argument("--disable-component-extensions-with-background-pages")
options.add_argument("--disable-default-apps")
options.add_argument("--enable-features=NetworkService,NetworkServiceInProcess")
options.add_argument("--disable-javascript-harmony-shipping")
options.add_argument("--disable-site-isolation-trials")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--disable-web-security")
options.add_argument("--disable-features=IsolateOrigins,site-per-process")
options.add_argument("--aggressive-cache-discard")
options.add_argument("--disable-cache")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-offline-load-stale-cache")
options.add_argument("--disk-cache-size=0")
options.page_load_strategy = 'none'  # Changed from 'eager' to 'none' for fastest possible loading

# Configure parallel processing
NUM_WORKERS = 20  # Increased workers
CHUNK_SIZE = max(1, len(df_players) // (NUM_WORKERS * 2))
SAVE_FREQUENCY = 25  # Save every 25 players

# Minimum viable timeouts (in seconds)
PAGE_LOAD_TIMEOUT = 2
SCRIPT_TIMEOUT = 2
ELEMENT_TIMEOUT = 2

# XPath constants
TABLE_XPATH = "/html/body/div[2]/section/div/section/div[5]/section/div[1]/div[6]/div[2]/div/table"
THEAD_XPATH = f"{TABLE_XPATH}/thead/tr"
TBODY_XPATH = f"{TABLE_XPATH}/tbody"

class PlayerDataExtractor:
    def __init__(self):
        self.driver = None
        self.wait = None
        
    async def setup(self):
        self.driver = webdriver.Chrome(options=options)
        self.driver.set_page_load_timeout(PAGE_LOAD_TIMEOUT)
        self.driver.set_script_timeout(SCRIPT_TIMEOUT)
        self.wait = WebDriverWait(self.driver, ELEMENT_TIMEOUT)
        
    async def cleanup(self):
        if self.driver:
            self.driver.quit()
            
    async def get_table_data(self, player_id: str, player_name: str) -> List[Dict]:
        try:
            url = f"https://baseballsavant.mlb.com/savant-player/{player_id}?stats=statcast-r-hitting-mlb"
            try:
                self.driver.get(url)
            except TimeoutException:
                # If timeout occurs, the page might still be usable
                pass
                
            try:
                table = self.wait.until(EC.presence_of_element_located((By.XPATH, TABLE_XPATH)))
            except TimeoutException:
                # One more quick attempt with a shorter timeout
                self.wait = WebDriverWait(self.driver, 1)
                table = self.wait.until(EC.presence_of_element_located((By.XPATH, TABLE_XPATH)))
            
            # Extract headers immediately without waiting
            header_map = {}
            header_elements = self.driver.find_elements(By.XPATH, f"{THEAD_XPATH}/th")
            for idx, elem in enumerate(header_elements):
                try:
                    header = elem.find_element(By.TAG_NAME, "div").text.strip()
                except:
                    header = elem.text.strip()
                header = header.replace("\n", " ").strip()
                if header in COLUMNS:
                    header_map[idx] = header
            
            # Extract rows without explicit wait
            tbody = self.driver.find_element(By.XPATH, TBODY_XPATH)
            rows = tbody.find_elements(By.TAG_NAME, "tr")
            
            player_rows = []
            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if not cells:
                    continue
                
                row_data = {col: "" for col in COLUMNS}
                row_data["Player ID"] = player_id
                has_data = False
                
                for idx, cell in enumerate(cells):
                    if idx in header_map:
                        value = cell.text.strip().replace(",", "")
                        if value:
                            row_data[header_map[idx]] = value
                            has_data = True
                
                if has_data:
                    player_rows.append(row_data)
            
            return player_rows
            
        except Exception as e:
            print(f"Error processing {player_name}: {str(e)}")
            return []

async def process_chunk(chunk_df, worker_id, progress_bar):
    global players_processed, all_rows
    extractor = PlayerDataExtractor()
    chunk_rows = []
    
    try:
        await extractor.setup()
        
        for _, row in chunk_df.iterrows():
            try:
                player_rows = await extractor.get_table_data(row["Player ID"], row["Player Name"])
                if player_rows:
                    chunk_rows.extend(player_rows)
                players_processed += 1
                progress_bar.update(1)
                
                # Save progress periodically
                if players_processed % SAVE_FREQUENCY == 0:
                    await save_progress(chunk_rows)
                    
            except Exception as e:
                print(f"Error in worker {worker_id}: {str(e)}")
                continue
            
        return chunk_rows
    finally:
        await extractor.cleanup()

async def save_progress(new_rows):
    if not new_rows:
        return
        
    temp_df = pd.DataFrame(all_rows + new_rows)
    if not temp_df.empty:
        temp_df = temp_df[COLUMNS]
        temp_df.to_csv(output_csv, index=False)

async def main():
    chunks = [df_players[i:i + CHUNK_SIZE] for i in range(0, len(df_players), CHUNK_SIZE)]
    print(f"\n🔍 Processing {len(df_players)} players using {NUM_WORKERS} workers ({len(chunks)} chunks)...\n")
    
    # Create progress bar for total players
    progress_bar = tqdm(total=len(df_players), desc="Processing Players")
    
    # Create semaphore to limit concurrent Chrome instances
    sem = asyncio.Semaphore(NUM_WORKERS)
    
    async def process_chunk_with_semaphore(chunk_df, worker_id):
        async with sem:
            return await process_chunk(chunk_df, worker_id, progress_bar)
    
    # Process chunks concurrently
    tasks = [
        asyncio.create_task(process_chunk_with_semaphore(chunk_df, i))
        for i, chunk_df in enumerate(chunks)
    ]
    
    # Wait for all tasks to complete
    chunk_results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Combine results
    for result in chunk_results:
        if isinstance(result, list):
            all_rows.extend(result)
    
    progress_bar.close()
    return all_rows

if __name__ == "__main__":
    try:
        start_time = time.time()
        all_rows = asyncio.run(main())
        
        if all_rows:
            df = pd.DataFrame(all_rows)
            df = df[COLUMNS]
            df.to_csv(output_csv, index=False)
            
            elapsed_time = time.time() - start_time
            print(f"\n✅ Completed in {elapsed_time:.2f} seconds")
            print(f"✅ Processed {players_processed} players")
            print(f"✅ Saved {len(df)} rows to {output_csv}")
        else:
            print("\n⚠️ No data was collected")
            
    except KeyboardInterrupt:
        print("\n\n⚠️ Process interrupted by user. Saving collected data...")
        if all_rows:
            df = pd.DataFrame(all_rows)
            df = df[COLUMNS]
            df.to_csv(output_csv, index=False)
            print(f"Saved {len(df)} rows to {output_csv}")
            
            
!open data/player_pitch_tracking_stats.csv



🔍 Processing 1454 players using 20 workers (41 chunks)...



Processing Players:   0%|          | 1/1454 [00:03<1:32:30,  3.82s/it]

Error processing Andrew Abbott: Message: 
Stacktrace:
0   chromedriver                        0x0000000100982dec cxxbridge1$str$ptr + 2817040
1   chromedriver                        0x000000010097b088 cxxbridge1$str$ptr + 2784940
2   chromedriver                        0x00000001004c28d8 cxxbridge1$string$len + 93028
3   chromedriver                        0x00000001005096a0 cxxbridge1$string$len + 383276
4   chromedriver                        0x000000010054a7b8 cxxbridge1$string$len + 649796
5   chromedriver                        0x00000001004fda80 cxxbridge1$string$len + 335116
6   chromedriver                        0x0000000100947c74 cxxbridge1$str$ptr + 2575000
7   chromedriver                        0x000000010094af40 cxxbridge1$str$ptr + 2588004
8   chromedriver                        0x00000001009279fc cxxbridge1$str$ptr + 2443296
9   chromedriver                        0x000000010094b7bc cxxbridge1$str$ptr + 2590176
10  chromedriver                        0x0000000100918af0 

Processing Players:   0%|          | 3/1454 [00:07<54:47,  2.27s/it]  

Error processing Bryan Abreu: Message: 
Stacktrace:
0   chromedriver                        0x0000000100982dec cxxbridge1$str$ptr + 2817040
1   chromedriver                        0x000000010097b088 cxxbridge1$str$ptr + 2784940
2   chromedriver                        0x00000001004c28d8 cxxbridge1$string$len + 93028
3   chromedriver                        0x00000001005096a0 cxxbridge1$string$len + 383276
4   chromedriver                        0x000000010054a7b8 cxxbridge1$string$len + 649796
5   chromedriver                        0x00000001004fda80 cxxbridge1$string$len + 335116
6   chromedriver                        0x0000000100947c74 cxxbridge1$str$ptr + 2575000
7   chromedriver                        0x000000010094af40 cxxbridge1$str$ptr + 2588004
8   chromedriver                        0x00000001009279fc cxxbridge1$str$ptr + 2443296
9   chromedriver                        0x000000010094b7bc cxxbridge1$str$ptr + 2590176
10  chromedriver                        0x0000000100918af0 cx

...

Error processing Ty Adcock: Message: 
Stacktrace:
0   chromedriver                        0x0000000100982dec cxxbridge1$str$ptr + 2817040
1   chromedriver                        0x000000010097b088 cxxbridge1$str$ptr + 2784940
2   chromedriver                        0x00000001004c28d8 cxxbridge1$string$len + 93028
3   chromedriver                        0x00000001005096a0 cxxbridge1$string$len + 383276
4   chromedriver                        0x000000010054a7b8 cxxbridge1$string$len + 649796
5   chromedriver                        0x00000001004fda80 cxxbridge1$string$len + 335116
6   chromedriver                        0x0000000100947c74 cxxbridge1$str$ptr + 2575000
7   chromedriver                        0x000000010094af40 cxxbridge1$str$ptr + 2588004
8   chromedriver                        0x00000001009279fc cxxbridge1$str$ptr + 2443296
9   chromedriver                        0x000000010094b7bc cxxbridge1$str$ptr + 2590176
10  chromedriver                        0x0000000100918af0 cxxb

Processing Players:   1%|          | 15/1454 [00:28<44:37,  1.86s/it]

Error processing Joan Adon: Message: 
Stacktrace:
0   chromedriver                        0x0000000100982dec cxxbridge1$str$ptr + 2817040
1   chromedriver                        0x000000010097b088 cxxbridge1$str$ptr + 2784940
2   chromedriver                        0x00000001004c28d8 cxxbridge1$string$len + 93028
3   chromedriver                        0x00000001005096a0 cxxbridge1$string$len + 383276
4   chromedriver                        0x000000010054a7b8 cxxbridge1$string$len + 649796
5   chromedriver                        0x00000001004fda80 cxxbridge1$string$len + 335116
6   chromedriver                        0x0000000100947c74 cxxbridge1$str$ptr + 2575000
7   chromedriver                        0x000000010094af40 cxxbridge1$str$ptr + 2588004
8   chromedriver                        0x00000001009279fc cxxbridge1$str$ptr + 2443296
9   chromedriver                        0x000000010094b7bc cxxbridge1$str$ptr + 2590176
10  chromedriver                        0x0000000100918af0 cxxb

Processing Players:   1%|          | 17/1454 [00:32<49:24,  2.06s/it]

Error processing Julian Aguiar: Message: 
Stacktrace:
0   chromedriver                        0x0000000100982dec cxxbridge1$str$ptr + 2817040
1   chromedriver                        0x000000010097b088 cxxbridge1$str$ptr + 2784940
2   chromedriver                        0x00000001004c28d8 cxxbridge1$string$len + 93028
3   chromedriver                        0x00000001005096a0 cxxbridge1$string$len + 383276
4   chromedriver                        0x000000010054a7b8 cxxbridge1$string$len + 649796
5   chromedriver                        0x00000001004fda80 cxxbridge1$string$len + 335116
6   chromedriver                        0x0000000100947c74 cxxbridge1$str$ptr + 2575000
7   chromedriver                        0x000000010094af40 cxxbridge1$str$ptr + 2588004
8   chromedriver                        0x00000001009279fc cxxbridge1$str$ptr + 2443296
9   chromedriver                        0x000000010094b7bc cxxbridge1$str$ptr + 2590176
10  chromedriver                        0x0000000100918af0 

In [ ]:
# # Batted Ball Profile Table
# # Quality Of Contact Table


# # Input and output paths
# input_folder = Path("data/raw/players-batting")  # folder with .html player files
# output_profile = "data/player_batted_ball_profile.csv"
# output_quality = "data/player_quality_of_contact.csv"

# all_profile_rows = []
# all_quality_rows = []

# # Helper function to parse tables
# def parse_table(table, slug):
#     headers = [th.get_text(strip=True) for th in table.find("thead").find_all("th")]
#     rows = []
#     for tr in table.find("tbody").find_all("tr"):
#         cells = tr.find_all(["td", "th"])
#         values = [td.get_text(strip=True) for td in cells]
#         if len(values) == len(headers):
#             row_dict = dict(zip(headers, values))
#             row_dict["Player Slug"] = slug
#             rows.append(row_dict)
#     return rows

# # Run test on uploaded files
# html_files = [
#     ("aaron-judge-592450.html", "aaron-judge-592450"),
#     ("brandon-lowe-664040.html", "brandon-lowe-664040"),
# ]

# for filename, slug in html_files:
#     with open(f"/mnt/data/{filename}", "r", encoding="utf-8") as f:
#         soup = BeautifulSoup(f, "html.parser")

#     tables = soup.find_all("table")
#     if len(tables) >= 2:
#         all_profile_rows.extend(parse_table(tables[0], slug))
#         all_quality_rows.extend(parse_table(tables[1], slug))

# # Convert to DataFrames
# df_profile = pd.DataFrame(all_profile_rows)
# df_quality = pd.DataFrame(all_quality_rows)

# # Show output here for verification
# import ace_tools as tools; tools.display_dataframe_to_user(name="Batted Ball Profile Table", dataframe=df_profile)
# tools.display_dataframe_to_user(name="Quality of Contact Table", dataframe=df_quality)


In [ ]:
# # Batted Ball v2

# from bs4 import BeautifulSoup
# import pandas as pd
# from pathlib import Path
# from tqdm import tqdm

# input_folder = Path("Scrapers/data/raw/players-batting")
# output_file = Path("data/player_pitch_tracking_stats.csv")

# all_rows = []
# files_processed = 0
# files_with_data = 0

# # Keep track of the first found table's columns to use as reference
# reference_columns = []

# print(f"🔍 Scanning {input_folder} for player HTML files...\n")

# all_files = list(input_folder.glob("*.html"))
# print(f"Found {len(all_files)} HTML files")

# for file_path in tqdm(all_files, desc="Processing players"):
#     files_processed += 1
    
#     with open(file_path, "r", encoding="utf-8") as f:
#         soup = BeautifulSoup(f.read(), "html.parser")

#     player_id = file_path.stem
#     print(f"\nProcessing {player_id}...")

#     # First try the standard detailedPitches ID
#     table = soup.find("table", {"id": "detailedPitches"})

#     # If not found, look for h2 elements containing "Pitch Tracking" text and find the next table
#     if not table:
#         pitch_tracking_headers = soup.find_all(lambda tag: tag.name == "h2" and "Pitch Tracking" in tag.get_text())
#         for header in pitch_tracking_headers:
#             table = header.find_next("table")
#             if table:
#                 break

#     # If still not found, search for tables with specific column headers
#     if not table:
#         candidate_tables = soup.find_all("table")
#         for t in candidate_tables:
#             headers = [th.get_text(strip=True) for th in t.find_all("th")]
#             if "Pitch Type" in headers and "Year" in headers:
#                 table = t
#                 break

#     if not table:
#         print(f"❌ No pitch tracking table found for {player_id}")
#         continue

#     # Extract headers in their original order
#     header_cells = table.find_all("th")
#     headers = [cell.get_text(strip=True) for cell in header_cells]
    
#     # Store the first table's columns as reference if not already set
#     if not reference_columns and len(headers) > 0:
#         reference_columns = headers.copy()
#         print(f"📊 Using column order from first found table: {reference_columns}")

#     # Extract rows
#     rows_found = 0
#     for tr in table.find("tbody").find_all("tr") if table.find("tbody") else table.find_all("tr"):
#         cells = tr.find_all("td")
#         if not cells or len(cells) <= 1:
#             continue
            
#         # Skip "Show More Seasons" rows
#         if len(cells) == 1 and "Show More" in cells[0].get_text():
#             continue
            
#         # Skip note rows about years in reverse order
#         if len(cells) == 1 and "Note:" in cells[0].get_text():
#             continue

#         row_data = {"Player ID": player_id}
#         for i, cell in enumerate(cells):
#             if i < len(headers):
#                 span = cell.find("span")
#                 text = span.get_text(strip=True) if span else cell.get_text(strip=True)
                
#                 # Add to row data using original header names
#                 row_data[headers[i]] = text

#         # Only add rows that have actual data
#         if len(row_data) > 1:  # More than just Player ID
#             all_rows.append(row_data)
#             rows_found += 1

#     if rows_found > 0:
#         files_with_data += 1
#         print(f"✓ Extracted {rows_found} rows for {player_id}")
#     else:
#         print(f"⚠️ No rows found for {player_id}")
    
#     # Save every 20 files
#     if files_processed % 20 == 0 and all_rows:
#         # Create intermediate save
#         temp_df = pd.DataFrame(all_rows)
        
#         # Clean up numeric values
#         for col in temp_df.columns:
#             if temp_df[col].dtype == object:  # Only process string columns
#                 temp_df[col] = temp_df[col].astype(str).str.replace(",", "")
        
#         # Filter out rows where Year is "Player" or "MLB"
#         if "Year" in temp_df.columns:
#             temp_df = temp_df[~temp_df["Year"].isin(["Player", "MLB"])]
        
#         # Force column order to match original table
#         cols_to_use = ["Player ID"]
#         for col in reference_columns:
#             if col in temp_df.columns:
#                 cols_to_use.append(col)
        
#         # Add any columns that might be in the data but not in our reference
#         for col in temp_df.columns:
#             if col not in cols_to_use:
#                 cols_to_use.append(col)
                
#         temp_df = temp_df[cols_to_use]
            
#         # Save intermediate result
#         temp_df.to_csv(output_file, index=False)
#         print(f"💾 Intermediate save: {len(all_rows)} rows saved after processing {files_processed} files")

# print(f"\n🏁 Processing complete!")
# print(f"Files processed: {files_processed}")
# print(f"Files with pitch tracking data: {files_with_data}")
# print(f"Total rows collected: {len(all_rows)}")

# if all_rows:
#     # Create final DataFrame
#     df = pd.DataFrame(all_rows)
    
#     # Clean up numeric values
#     for col in df.columns:
#         if df[col].dtype == object:  # Only process string columns
#             df[col] = df[col].astype(str).str.replace(",", "")
    
#     # Filter out rows where Year is "Player" or "MLB"
#     if "Year" in df.columns:
#         df = df[~df["Year"].isin(["Player", "MLB"])]
    
#     # Force column order to match original table
#     cols_to_use = ["Player ID"]
#     for col in reference_columns:
#         if col in df.columns:
#             cols_to_use.append(col)
    
#     # Add any columns that might be in the data but not in our reference
#     for col in df.columns:
#         if col not in cols_to_use:
#             cols_to_use.append(col)
    
#     # Reorder columns according to our list
#     df = df[cols_to_use]
    
#     # Make sure output directory exists
#     output_file.parent.mkdir(parents=True, exist_ok=True)
    
#     df.to_csv(output_file, index=False)
#     print(f"💾 Final data saved to {output_file}")
#     print(f"Final column order: {list(df.columns)}")
    
#     if len(df) > 0:
#         print(f"Sample of extracted data:")
#         print(df.head(2))
# else:
#     print("⚠️ No pitch tracking data found to save")

# # import ace_tools as tools; tools.display_dataframe_to_user(name="Pitch Tracking Stats", dataframe=df)


In [ ]:
# import os
# import pandas as pd
# from bs4 import BeautifulSoup
# from pathlib import Path
# from tqdm import tqdm

# input_folder = Path("data/raw/players-batting")
# output_file = Path("data/player_pitch_tracking_stats.csv")

# all_rows = []
# all_columns = set()

# # Loop through all player HTML files
# for file_path in tqdm(list(input_folder.glob("*.html")), desc="Processing players"):
#     with open(file_path, "r", encoding="utf-8") as f:
#         soup = BeautifulSoup(f.read(), "html.parser")

#     player_id = file_path.stem

#     # Locate the Pitch Tracking table
#     table = soup.find("table", {"id": "detailedPitches"})
#     if not table:
#         continue

#     # Extract headers
#     header_cells = table.find("thead").find_all("th")
#     headers = [cell.get_text(strip=True) for cell in header_cells]
#     all_columns.update(headers)

#     # Extract rows
#     for tr in table.find("tbody").find_all("tr"):
#         cells = tr.find_all("td")
#         if not cells:
#             continue

#         row_data = {}
#         for i, cell in enumerate(cells):
#             # Grab text inside <span> or directly from cell
#             span = cell.find("span")
#             text = span.get_text(strip=True) if span else cell.get_text(strip=True)
#             col_name = headers[i] if i < len(headers) else f"Extra_{i}"
#             row_data[col_name] = text
#             all_columns.add(col_name)

#         row_data["Player ID"] = player_id
#         all_rows.append(row_data)

# # Ensure consistent columns
# final_columns = ["Player ID"] + sorted(all_columns - {"Player ID"})
# df = pd.DataFrame(all_rows, columns=final_columns)

# # Clean up numeric values (remove commas)
# for col in df.columns:
#     df[col] = df[col].str.replace(",", "", regex=False)

# df.to_csv(output_file, index=False)
# print(f"✅ Done. Saved pitch tracking stats to: {output_file}")



In [ ]:
# Scrape Pitching Stats 